# Last Month's Wind Forcing of the Ocean from Scatterometry
Ruth Moorman April 17th 2024 (for Ian Fenty ESE144 lectures)


In this notebook we'll be looking at some Level 3 processed scatterometry data from the ASCAT (Advanced Scatterometer) aboard MetOp-C, the most recent fo the MetOp Satellites launched in 2018 and expected to operate until at least 2027. I've sourced this data from the <a href="https://data.marine.copernicus.eu/product/WIND_GLO_PHY_L3_NRT_012_002/description">Copernicus Marine Data Store</a>, a European data repository I recommend you take a look at. 
<br><br>

This data contains gridded instantaneous measurements, so it's level of processing is still relavtively low despite being L3 data. The lack of temporal processing means the data can get quite large, so we'll just be looking at the latest full month of data, March 2024. Here are the direct links to the <a href="https://data.marine.copernicus.eu/product/WIND_GLO_PHY_L3_NRT_012_002/files?subdataset=cmems_obs-wind_glo_phy_nrt_l3-metopc-ascat-asc-0.25deg_P1D-i_202311&path=WIND_GLO_PHY_L3_NRT_012_002%2Fcmems_obs-wind_glo_phy_nrt_l3-metopc-ascat-asc-0.25deg_P1D-i_202311%2F2024%2F03%2F">ascending</a> and 
    <a href="https://data.marine.copernicus.eu/product/WIND_GLO_PHY_L3_NRT_012_002/files?subdataset=cmems_obs-wind_glo_phy_nrt_l3-metopc-ascat-des-0.25deg_P1D-i_202311&path=WIND_GLO_PHY_L3_NRT_012_002%2Fcmems_obs-wind_glo_phy_nrt_l3-metopc-ascat-des-0.25deg_P1D-i_202311%2F2024%2F03%2F">descending</a> swath data from that month, for your reference, however the data is stored locally so you don't need to download it now.


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature


<div class="alert alert-block alert-info">
<b>XARRAY:</b>
<div> 
<div>        
  
Since netCDF data structures are now incredibly widespread in rsatellite remote sensing and climate science more broadly, we'll be using the <a href="https://docs.xarray.dev/en/stable/">xarray</a> python package, a suite of functions built to interface smoothly with netCDF files, for our python work in this course.
    
    <br><br>
In this notebook we'll use the following xarray functions:
   <br> 
• <a href="https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html">xr.open_mfdataset()</a>: opens numerous sequential (here, sequential in time) netCDF files at once and organises them into a single manipulatable data structure called a Dataset (ds) containing numerous <a href="https://docs.xarray.dev/en/latest/user-guide/data-structures.html">DataArrays (da)</a>
   <br> 
• <a href="https://docs.xarray.dev/en/latest/user-guide/indexing.html">da.sel()</a>: allows you to index data according to the real value of the dimension (e.g. pull out information associated with a date or lat/lon position using that date or lat/lon value rather than an index)
  <br> 
• <a href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.count.html">da.count()</a>: counts the number of non-NaN values along a dimension
 <br> 
• <a href="https://docs.xarray.dev/en/stable/generated/xarray.concat.html">xr.concat()</a>: concatenates data along some dimension
  <br> 
• <a href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.coarsen.html">da.coarsen()</a>: coarsens data in some dimension
  <br> 
• <a href="https://docs.xarray.dev/en/stable/generated/xarray.where.html">da.where()</a>: masks out values not satisfying a listed criteria
    
    
    
<div>   
    

### Opening Data with `xr.open_mfdataset`

In [ ]:
## ascending swath data for March 2024
ds_asc = xr.open_mfdataset('~/shared/notebooks/CaltechESE1442024/data/metopc-scatterometry/GLO-WIND_L3-OBS_METOP-C_ASCAT_25_ASC*.nc')
## descending swath data for March 2024
ds_des = xr.open_mfdataset('~/shared/notebooks/CaltechESE1442024/data/metopc-scatterometry/GLO-WIND_L3-OBS_METOP-C_ASCAT_25_DES*.nc')

In [ ]:
ds_asc

In [ ]:
ds_des

### Extracting the `DataArray` variables we will use in this notebook

In [ ]:
time_asc           = ds_asc.measurement_time # measurement time from the ascending swath
time_des           = ds_des.measurement_time # measurement time from the descending swath
wind_speed_asc     = ds_asc.wind_speed       #wind speed from the ascending swath
wind_speed_des     = ds_des.wind_speed       #wind speed from the descending swath
eastward_wind_asc  = ds_asc.eastward_wind    #eastward wind velocity component from the ascending swath
eastward_wind_des  = ds_des.eastward_wind    #eastward wind velocity component from the descending swath
northward_wind_asc = ds_asc.northward_wind   #northward wind velocity component from the ascending swath
northward_wind_des = ds_des.northward_wind   #northward wind velocity component from the descending swath

## Plotting the data coverage for March 2024

Let's start by familiarizing ourselves with the data structure by looking at the ascending and descending swath paths for a single day (1st March).

In [ ]:
time_asc_plot = time_asc.sel(time="2024-03-01").squeeze("time")
time_des_plot = time_des.sel(time="2024-03-01").squeeze("time")

In [ ]:
fig, (ax, ax1) = plt.subplots(1,2, figsize=(20,5), gridspec_kw={'wspace':0.1},subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
ax.add_feature(cartopy.feature.COASTLINE);ax1.add_feature(cartopy.feature.COASTLINE)

asc = time_asc_plot.astype('float'); asc = asc.where(asc>0); asc = (asc - asc.min())/(asc.max()- asc.min()); 
c = ax.pcolormesh(time_asc.lon, time_asc.lat, asc, transform = ccrs.PlateCarree(), cmap='viridis', vmin=0, vmax=1)

des = time_des_plot.astype('float'); des = des.where(des>0); des = (des - des.min())/(des.max()- des.min()); 
ax1.pcolormesh(time_des.lon, time_des.lat, des, transform = ccrs.PlateCarree(), cmap='viridis', vmin=0, vmax=1)

ax.set_title('Ascending Swath Measurement Time (fraction of day, 1 March 2024)');ax1.set_title('Descending Swath Measurement Time (fraction of day, 1 March 2024)')
cax = fig.add_axes([0.08, 0.2, 0.015, 0.6])
cbar=plt.colorbar(c,cax = cax,orientation='vertical',shrink = 0.5)
plt.show()


<div class="alert alert-block alert-info">
<b>DISCUSS:</b>
<div> 
<div>        
  
Based on this orbit path, where do you expect relatively high and low data coverage over the course of a month?    
    
<div>   
    


<div class="alert alert-block alert-info">
<b>CODING:</b>
<div> 
<div>        
  
Use xarray's count function to determine how many data values are found within each grid cell over the course of the month. Use the <b>wind_speed_asc</b> and <b>wind_speed_des</b> DataArrays (da, in documentation) for this.    
<div>   
    

In [ ]:
measurement_count_asc      =  ## YOUR CODE ##
measurement_count_des      =  ## YOUR CODE ##
measurement_count_combined = measurement_count_asc + measurement_count_des

In [ ]:
fig, (ax, ax1) = plt.subplots(1,2, figsize=(20,5), gridspec_kw={'wspace':0.1},subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
c = ax.pcolormesh(measurement_count_asc.lon, measurement_count_asc.lat, measurement_count_asc, transform = ccrs.PlateCarree(), cmap='viridis', vmin=0, vmax=31)
ax1.pcolormesh(measurement_count_des.lon, measurement_count_des.lat, measurement_count_des, transform = ccrs.PlateCarree(), cmap='viridis', vmin=0, vmax=31)
ax.set_title('Ascending Swath Measurement Count (N days with data, March 2024)');ax1.set_title('Descending Swath Measurement Count (N days with data, March 2024)')
ax.set_xticks(np.arange(-180,190,30));ax.set_yticks(np.arange(-80,90,20))
ax1.set_xticks(np.arange(-180,190,30));ax1.set_yticks(np.arange(-80,90,20))
cax = fig.add_axes([0.09, 0.2, 0.01, 0.6])
cbar=plt.colorbar(c,cax = cax,orientation='vertical',shrink = 0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,7), gridspec_kw={'wspace':0.1},subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
c = ax.pcolormesh(measurement_count_combined.lon, measurement_count_combined.lat, measurement_count_combined, transform = ccrs.PlateCarree(), cmap='viridis', vmin=0, vmax=62)
ax.set_title('Swath Measurement Count (N data per gridcell, March 2024)')
ax.set_xticks(np.arange(-180,190,30));ax.set_yticks(np.arange(-80,90,20))
cax = fig.add_axes([0.07, 0.2, 0.015, 0.6])
cbar=plt.colorbar(c,cax = cax,orientation='vertical',shrink = 0.5)
plt.show()


<div class="alert alert-block alert-info">
<b>DISCUSS:</b>
<div> 
<div>        
  
What are some potential advantages and disadvantages to this resolution pattern? 
<div>   
    

## Plotting the mean wind speed at 10 m for March 2024


<div class="alert alert-block alert-info">
<b>DISCUSS:</b>
<div> 
<div>        
  
Below I concatenate the ascending and descending swath data before taking their time average, why do I need to do this (or something similar)?
    <div>   
    

In [ ]:
# cannot directly concatenate along the time dimension as there are identical between the DataArrays
wind_speed_asc_concat = wind_speed_asc.assign_coords({'time': np.arange(31)})   .rename({"time":"concat_index"})      
wind_speed_des_concat = wind_speed_des.assign_coords({'time': np.arange(31,62)}).rename({"time":"concat_index"})      

######
mean_wind_speed = xr.concat([wind_speed_asc_concat,wind_speed_des_concat], dim='concat_index').mean('concat_index')
######

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,7),subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})

c = ax.pcolormesh(mean_wind_speed.lon, mean_wind_speed.lat, mean_wind_speed, transform = ccrs.PlateCarree(), cmap='jet', vmin=3, vmax=15)

ax.set_title('March 2024 mean 10m wind speed [m/s]')
ax.set_xticks(np.arange(-180,190,30));ax.set_yticks(np.arange(-80,90,20))
cax = fig.add_axes([0.07, 0.2, 0.015, 0.6])
cbar=plt.colorbar(c,cax = cax,orientation='vertical',shrink = 0.5)
plt.show()


<div class="alert alert-block alert-info">
<b>DISCUSS:</b>
<div> 
<div>        
  
What are some notable features in the wind speed field? How do last months winds compare to the NH Summer and NH Winter fields shown in the lecture? 
    <br>
This is a pretty crude time averaging, in what circumstances might you want to smooth this data?
    <div>   
    

## Plotting the mean wind velocity (zonal and meridional components) at 10 m for March 2024


<div class="alert alert-block alert-info">
<b>CODING:</b>
<div> 
<div>        
  
In a similar fashion to the above computation of mean wind speed (i.e. you can copy/paste/modify), compute the March 2024 mean eastward wind (call this "mean_u") and northward wind (call this "mean_v").<div>   
    

In [ ]:
## your code

In [ ]:
fig, (ax, ax1) = plt.subplots(1,2, figsize=(20,5), gridspec_kw={'wspace':0.1},subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
# c = ax .pcolormesh(mean_u.lon, mean_u.lat, mean_u, transform = ccrs.PlateCarree(), cmap='jet', vmin=0, vmax=12)
# c = ax1.pcolormesh(mean_v.lon, mean_v.lat, mean_v, transform = ccrs.PlateCarree(), cmap='jet', vmin=0, vmax=12)
ax .set_title('March 2024 mean 10m u [m/s]')
ax1.set_title('March 2024 mean 10m v [m/s]')
ax.set_xticks(np.arange(-180,190,30));ax.set_yticks(np.arange(-80,90,20))
ax1.set_xticks(np.arange(-180,190,30));ax1.set_yticks(np.arange(-80,90,20))
cax = fig.add_axes([0.07, 0.2, 0.008, 0.6])
cbar=plt.colorbar(c,cax = cax,orientation='vertical',shrink = 0.5)
plt.show()


<div class="alert alert-block alert-info">
<b>CODING:</b>
<div> 
<div>        
  
Modify the degree of horizontal coarsening in the da.coarsen() function below until directional wind structures are visible.<div>   
    

In [ ]:
fig = plt.figure(figsize=[15, 7])
proj = ccrs.PlateCarree(central_longitude=180)
ax  = plt.subplot(projection = proj)
c = ax.pcolormesh(mean_wind_speed.lon, mean_wind_speed.lat, mean_wind_speed, transform = ccrs.PlateCarree(), cmap='jet', vmin=3, vmax=15)

w=2 ## change this value
coarse_mean_u = mean_u.coarsen(lon=w,lat=w, boundary='pad').mean()
coarse_mean_v = mean_v.coarsen(lon=w,lat=w, boundary='pad').mean()

lon, lat = np.meshgrid(coarse_mean_u.lon,coarse_mean_u.lat)
ax.quiver(x=lon,y=lat , u=coarse_mean_u.values, v=coarse_mean_v.values, transform = ccrs.PlateCarree())

ax.set_xticks(np.arange(-180,190,30));ax.set_yticks(np.arange(-80,90,20))
cax = fig.add_axes([0.07, 0.2, 0.015, 0.6])
cbar=plt.colorbar(c,cax = cax,orientation='vertical',shrink = 0.5)
plt.show()



<div class="alert alert-block alert-info">
<b>DISCUSS:</b>
<div> 
<div>        
  
What are some interesting features revealed by the direction of the wind? What directions are winds going relative to coastlines, and why does this matter?    <div>   
    